In [1]:
import pandas as pd

df_single = pd.read_excel('data/single.xlsx')
df_choose = pd.read_excel('data/choose.xlsx')
df_single_4o = pd.read_excel('data/single_4o.xlsx')
df_choose_4o = pd.read_excel('data/choose_4o.xlsx')
print(df_single)
print(df_single_4o)

    id                            q  \
0    1         你是如何根据客户的风险特征来确定保费的？   
1    2       你能举一个例子说明如何使用风险分类来定价吗？   
2    3          请解释如何根据历史损失数据来设定保费。   
3    4          在实际操作中，如何平衡纯保费和总保费？   
4    5       你如何在定价中考虑客户的内在风险和外在风险？   
5    6         一个客户的保单生命周期如何影响定价策略？   
6    7          请分享一个你处理过的损失分布模型案例。   
7    8         请描述你处理过的一次复杂的保险定价策略。   
8    9           你是如何确定一款新产品的定价策略的？   
9   10         请举例说明如何根据市场竞争情况调整定价。   
10  11          你如何使用市场调研数据来做出定价决策？   
11  12  请分享一个你如何通过调整定价策略来提升市场份额的实例。   
12  13      在面临市场波动时，你如何调整定价以保持稳定性？   
13  14          你如何在定价策略中兼顾盈利性和竞争力？   
14  15          请描述你在实际工作中如何进行风险评估。   
15  16        你能举一个实际案例说明如何进行风险分层吗？   
16  17     请分享一个你处理过的涉及道德风险和逆选择的案例。   
17  18           你是如何评估再保险需求并确定定价的？   
18  19      请解释一个你利用灾害建模来确定定价的实际案例。   
19  20           在评估客户风险时，你会考虑哪些因素？   
20  21      请描述你在定价过程中遇到的合规挑战及解决办法。   
21  22        如何在实际操作中确保定价过程符合监管要求？   
22  23         请举例说明法规变化如何影响你的定价策略。   
23  24           你如何处理监管机构的反馈并调整定价？   
24  25        请描述你在定价过程中如

In [6]:
# 分别计算answer_k和answer_n的主题，构建LDAModel
import jieba
from gensim import corpora
from gensim.models import LdaModel
import string

# 读取中文停用词表
with open('hit_stopwords.txt', 'r', encoding='utf-8') as file:
    stop_words = set(file.read().splitlines())
    
# 提取所需列
texts_k = df_single['answer_k'].astype(str).tolist()
texts_n = df_single['answer_n'].astype(str).tolist()

def preprocess_text(text):
    # 去掉标点符号
    text = text.translate(str.maketrans('', '', string.punctuation + '，。！？；：（）【】'))
    # 分词
    words = jieba.cut(text)
    # 去停用词
    words = [word for word in words if word not in stop_words and len(word) > 1]
    return words

# 预处理文本数据
processed_texts_k = [preprocess_text(text) for text in texts_k]
processed_texts_n = [preprocess_text(text) for text in texts_n]

# 构建词典和语料库
dictionary_k = corpora.Dictionary(processed_texts_k)
corpus_k = [dictionary_k.doc2bow(text) for text in processed_texts_k]

dictionary_n = corpora.Dictionary(processed_texts_n)
corpus_n = [dictionary_n.doc2bow(text) for text in processed_texts_n]

# 训练LDA模型
num_topics = 5  # 主题数，可以根据需求调整
lda_model_k = LdaModel(corpus=corpus_k, id2word=dictionary_k, num_topics=num_topics, random_state=42)
lda_model_n = LdaModel(corpus=corpus_n, id2word=dictionary_n, num_topics=num_topics, random_state=42)

# 提取主题和关键词
def get_document_topics_and_keywords(model, corpus):
    topics = [model.get_document_topics(doc) for doc in corpus]
    keywords = model.show_topics(num_topics=num_topics, num_words=10, formatted=False)
    return topics, keywords

topics_k, keywords_k = get_document_topics_and_keywords(lda_model_k, corpus_k)
topics_n, keywords_n = get_document_topics_and_keywords(lda_model_n, corpus_n)

# 输出主题和关键词
print("Topics and keywords for answer_k:")
for i, topics in enumerate(topics_k):
    print(f"Document {i} topics:")
    for topic in topics:
        print(f"Topic {topic[0]}, weight: {topic[1]}")
    print("Topic keywords:")
    for topic_id, topic_keywords in keywords_k:
        if any(t[0] == topic_id for t in topics):
            print(f"Topic {topic_id} keywords: {', '.join([word for word, _ in topic_keywords])}")

print("\nTopics and keywords for answer_n:")
for i, topics in enumerate(topics_n):
    print(f"Document {i} topics:")
    for topic in topics:
        print(f"Topic {topic[0]}, weight: {topic[1]}")
    print("Topic keywords:")
    for topic_id, topic_keywords in keywords_n:
        if any(t[0] == topic_id for t in topics):
            print(f"Topic {topic_id} keywords: {', '.join([word for word, _ in topic_keywords])}")


Topics and keywords for answer_k:
Document 0 topics:
Topic 3, weight: 0.9969556331634521
Topic keywords:
Topic 3 keywords: 风险, 驾驶, 行为, 模型, 数据, 保险公司, 保费, 客户, 定价, 评估
Document 1 topics:
Topic 3, weight: 0.9975545406341553
Topic keywords:
Topic 3 keywords: 风险, 驾驶, 行为, 模型, 数据, 保险公司, 保费, 客户, 定价, 评估
Document 2 topics:
Topic 3, weight: 0.9973542094230652
Topic keywords:
Topic 3 keywords: 风险, 驾驶, 行为, 模型, 数据, 保险公司, 保费, 客户, 定价, 评估
Document 3 topics:
Topic 3, weight: 0.9967646598815918
Topic keywords:
Topic 3 keywords: 风险, 驾驶, 行为, 模型, 数据, 保险公司, 保费, 客户, 定价, 评估
Document 4 topics:
Topic 3, weight: 0.9974045753479004
Topic keywords:
Topic 3 keywords: 风险, 驾驶, 行为, 模型, 数据, 保险公司, 保费, 客户, 定价, 评估
Document 5 topics:
Topic 3, weight: 0.9965474009513855
Topic keywords:
Topic 3 keywords: 风险, 驾驶, 行为, 模型, 数据, 保险公司, 保费, 客户, 定价, 评估
Document 6 topics:
Topic 2, weight: 0.015717245638370514
Topic 3, weight: 0.9823540449142456
Topic keywords:
Topic 2 keywords: 定价, 驾驶, 客户, 数据, 价格, 策略, 市场, 行为, 风险, 产品
Topic 3 keywords: 风险